# Construyendo un chatbot con NLTK desde cero

## Importamos librerías

In [1]:
import io
import random
import string
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

## Importamos NLTK 

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/gustavo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Leemos el corpus con el texto para el chatbot

Podemos utilizar una página de Wikipedia para crear el corpus. Podemos copiar el contenido en un fichero de texto.

Debemos procesar el texto para para convertirlo en datos que el algoritmo pueda interpretar. Por ejemplo:

* Convertir el texto a mayúsculas/minúsculas para que el algoritmo no diferencie los token.
* Tokenización: NLTK permite tokenizar por frases (sentence tokenizer) y por palabras (word tokenizer)

El paquete NLTK incluye:

* Listas de stopwords
* Stemming: obtener la raíz de una palabra. Por ejemplo, el algoritmo de Porter es un algoritmo de derivación de palabras y consiste en elimar las términaciones morfológicas comunes de las palabras en inglés, aunque es posible usarlo también en español. Es un proceso de normalización de términos muy usado en corpus basados en texto. Por ejemplo, texto lo transforma a text y canta a cant.
* Lematización: relaciona una palabra flexionada o derivada con su forma canónica o lema. Por ejemplo, cantas, canto lo convierte a cantar. 

In [3]:
f=open('chatbot-es.txt','r',errors = 'ignore')
raw=f.read()
raw = raw.lower()# converts to lowercase
print (raw)

un bot de charla o bot conversacional (en inglés, chatbot) es un programa que simula mantener una conversación con una persona al proveer respuestas automáticas a entradas hechas por el usuario. habitualmente, la conversación se establece mediante texto, aunque también hay modelos que disponen de una interfaz de usuario multimedia. más recientemente, algunos comienzan a utilizar programas conversores de texto a sonido (ctv), dotándolo de mayor realismo a la interacción con el usuario.

para establecer una conversación han de utilizarse frases fácilmente comprensibles y que sean coherentes, aunque la mayoría de los bot conversacionales no consiguen comprender del todo. en su lugar, tienen en cuenta las palabras o frases del interlocutor, que les permitirán usar una serie de respuestas preparadas de antemano. de esta manera, el bot es capaz de seguir una conversación con más o menos lógica, pero sin saber realmente de qué está hablando.

los chatbots tienen su origen en 1966, cuando jose

## Tokenization

In [4]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words
print (sent_tokens)

['un bot de charla o bot conversacional (en inglés, chatbot) es un programa que simula mantener una conversación con una persona al proveer respuestas automáticas a entradas hechas por el usuario.', 'habitualmente, la conversación se establece mediante texto, aunque también hay modelos que disponen de una interfaz de usuario multimedia.', 'más recientemente, algunos comienzan a utilizar programas conversores de texto a sonido (ctv), dotándolo de mayor realismo a la interacción con el usuario.', 'para establecer una conversación han de utilizarse frases fácilmente comprensibles y que sean coherentes, aunque la mayoría de los bot conversacionales no consiguen comprender del todo.', 'en su lugar, tienen en cuenta las palabras o frases del interlocutor, que les permitirán usar una serie de respuestas preparadas de antemano.', 'de esta manera, el bot es capaz de seguir una conversación con más o menos lógica, pero sin saber realmente de qué está hablando.', 'los chatbots tienen su origen en

## Preprocesando

Analizamos los tokens para normalizarlos

In [5]:
stemmer = SnowballStemmer('spanish')

def LemTokens(tokens):
    #return [lemmer.lemmatize(token) for token in tokens]
    return [stemmer.stem(token) for token in tokens]

# diccionario con caracteres puntuacion
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


print(string.punctuation)
print(remove_punct_dict)
print("Gustavo\" juega al? fútbol!".lower().translate(remove_punct_dict))
print(LemNormalize("Gustavo\" juega al? fútbol!"))

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}
gustavo juega al fútbol
['gustav', 'jueg', 'al', 'futbol']


Definimos una función para saludar.

In [6]:
GREETING_INPUTS = ("hola", "que tal?")
GREETING_RESPONSES = ["Buenos días", "hola"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

### Generamos la respuesta

### Bag of Words

Después del preproceso inicial, transformamos el texto en un vector (array) de números. El bag-of-words es la representación del texto que describe las ocurrencias de las palabras en un documento incluyendo el vocabulario de palabras y el número de ocurrencias. No se tiene en cuenta ni dónde ocurren ni el orden, solo tiene en cuenta si las palabras aparecen en el texto.

En el caso de bag-of-words los documentos se consideran similares si contienen texto similar. Por ejemplo, {Gustavo, juega, al, fútbol} y vectorizamos el texto "Gustavo juega", tendríamos el siguiente vector: (1,1,0,0).


**TF-IDF**

Term Frequency: la frecuenca de las palabras en un documento.

TF = (Número de veces que el término t aparece en un documento)/(Número de términos del documento)

Inverse Document Frequency: mide la relevancia de una palabra para un documento en una colección.

IDF = 1+log(N/n), donde, N el número de documentos y n es el número de documentos donde el térmimo t aparece.


**Similitud del coseno**
Es una medida de la similitud existente entre dos vectores. Mide el ángulo entre dos vectores.



Para generar una respuesta los conceptos de similitud se tienen en cuenta. Definimos una función de respuesta que busca una o más palabras y devuelve una posible respuesta. Si no encuentra nada devuelve: ”Lo siento! No entiendo tu pregunta”

In [7]:
ejemplo = "un bot de charla o bot conversacional (en inglés, chatbot) es un programa que simula mantener una conversación con una persona al proveer respuestas automáticas a entradas hechas por el usuario."
sent_tokens_ejemplo = nltk.sent_tokenize(ejemplo)# converts to list of sentences 
sent_tokens.append("Que es un chatbot?")

print(sent_tokens_ejemplo);

# Definimos tfidf
TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words("spanish"))
tfidf = TfidfVec.fit_transform(sent_tokens)

#print(sent_tokens)
#print(tfidf[-1])
print("con el indice -1 accedemos al ultimo valor del vector")
#print(tfidf[1])

# tfidf almacena para cada frase, los tokens y su valor
#print(tfidf)

# calculamos similitud coseno comparando la ultima posicion y el array completo
vals = cosine_similarity(tfidf[-1], tfidf)
print(vals)
print(vals[0])
#print(vals.argsort()[0])
print(vals.argsort()[0])
idx=vals.argsort()[0][-1]
print(idx)


['un bot de charla o bot conversacional (en inglés, chatbot) es un programa que simula mantener una conversación con una persona al proveer respuestas automáticas a entradas hechas por el usuario.']
con el indice -1 accedemos al ultimo valor del vector
[[0.22690126 0.         0.         0.         0.         0.
  0.21089344 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.19940077 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.3205251  0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         1.        ]]
[0.22690126 0.         0.         0.         0.         0.
 0.21089344 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.     

In [8]:
arr = np.array([[1,2,3,4,5], [6,7,8,9,10]])

print('Ultima posicion segunda dimension: ', arr[1, -1])
print('Penultima posicion segunda dimension: ', arr[1, -2])

Ultima posicion segunda dimension:  10
Penultima posicion segunda dimension:  9


In [9]:
def response(user_response):
    robo_response=''
    
    # incorporamos pregunta al corpus
    sent_tokens.append(user_response)
    
    # Definimos tfidf
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words("spanish"))
    tfidf = TfidfVec.fit_transform(sent_tokens)
    
    # calculamos similitud coseno de la ultima posicion con el vector completo
    vals = cosine_similarity(tfidf[-1], tfidf)
    print(vals)
    print(vals[0])
    print(vals.argsort()[0])
    print(vals.argsort()[0][-2])
    
    # devuelve los indices que ordenan un array
    # en la posicion -1, ultima, se encuentra la pregunta del usuario que hemos añadido
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"Lo siento! No entiendo tu pregunta"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

Finalmente, añadimos una función para solicitar el texto por pantalla.

In [ ]:
flag=True
print("ATI-UA: Mi nombre is ATI-UA y puedo contestar preguntas relacionadas con chatbots. Si deseas salir, inserta Adios!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='adios'):
        if(user_response=='Gracias' or user_response=='Muchas gracias' ):
            flag=False
            print("ATI-UA: De nada.")
        else:
            if(greeting(user_response)!=None):
                print("ATI-UA: "+greeting(user_response))
            else:
                print("ATI-UA: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ATI-UA: Hasta luego")

Bot: Mi nombre is Bot y puedo contestar preguntas relacionadas con chatbots. Si deseas salir, inserta Adios!
chatbot
Bot: [[0.21679892 0.         0.         0.         0.         0.
  0.20156444 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.19051469 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.30719203 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         1.         1.        ]]
[0.21679892 0.         0.         0.         0.         0.
 0.20156444 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.19051469 0.         0.         0.         0.
 0.         0.         0.         0.         0.       

#  Referencias

* https://github.com/parulnith/Building-a-Simple-Chatbot-in-Python-using-NLTK/blob/master/Chatbot.ipynb
* https://medium.com/analytics-vidhya/building-a-simple-chatbot-in-python-using-nltk-7c8c8215ac6e
* [Intro NLP](http://josearcosaneas.github.io/python/r/procesamiento/lenguaje/2017/01/02/procesamiento-lenguaje-natural-0.html#:~:text=El%20stemming%20consiste%20en%20extreaer,ayuda%20de%20la%20librer%C3%ADa%20NLTK.)